In [6]:
import os
import dagshub
import mlflow

In [2]:
os.chdir("../")

In [3]:
%pwd

'e:\\AI-DA-Portfolio\\chest-cancer-classification'

In [18]:
dagshub.init(repo_owner='yousefmekhlafi', repo_name='end-to-end-chest-cancer-classification-mlflow-dvc', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

Initialized MLflow to track repo "yousefmekhlafi/end-to-end-chest-cancer-classification-mlflow-dvc"

[2024-08-24 21:43:34,885: INFO: helpers: Initialized MLflow to track repo "yousefmekhlafi/end-to-end-chest-cancer-classification-mlflow-dvc"]


Repository yousefmekhlafi/end-to-end-chest-cancer-classification-mlflow-dvc initialized!

[2024-08-24 21:43:34,893: INFO: helpers: Repository yousefmekhlafi/end-to-end-chest-cancer-classification-mlflow-dvc initialized!]
[2024-08-24 21:43:34,945: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'ProtocolError('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))': /yousefmekhlafi/end-to-end-chest-cancer-classification-mlflow-dvc.mlflow/api/2.0/mlflow/runs/create]


In [4]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/yousefmekhlafi/end-to-end-chest-cancer-classification-mlflow-dvc"
os.environ["MLFLOW_TRACKING_USERNAME"]="yousefmekhlafi"
os.environ["MLFLOW_TRACKING_PASSWORD"]="cef18e2fee5e75366f107327c19624f9699fe6b9"

In [7]:
mlflow.set_tracking_uri('https://dagshub.com/yousefmekhlafi/end-to-end-chest-cancer-classification-mlflow-dvc.mlflow')

In [8]:
dagshub.init(repo_owner='yousefmekhlafi', repo_name='end-to-end-chest-cancer-classification-mlflow-dvc', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

c:\Users\Abdoo\.conda\envs\ccancer\lib\site-packages\rich\live.py:229: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=0df607c6-a93d-4a3f-a0fb-e41977c1657d&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=22b148ca441c7f730c550a08796b5274c9a65135bf021e6b618b6d99b4cb6805




Accessing as yousefmekhlafi

Initialized MLflow to track repo "yousefmekhlafi/end-to-end-chest-cancer-classification-mlflow-dvc"

Repository yousefmekhlafi/end-to-end-chest-cancer-classification-mlflow-dvc initialized!

In [9]:
import tensorflow as tf

In [10]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [11]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [12]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [13]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chest-CT-Scan-data",
            mlflow_uri="https://dagshub.com/yousefmekhlafi/end-to-end-chest-cancer-classification-mlflow-dvc",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [14]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [15]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [ ]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e